--- Day 17: Conway Cubes ---

As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.

The experimental energy source is based on cutting-edge technology: a set of Conway Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.

The pocket dimension contains an infinite 3-dimensional grid. At every integer 3-dimensional coordinate (x,y,z), there exists a single cube which is either active or inactive.

In the initial state of the pocket dimension, almost all cubes start inactive. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified active (#) or inactive (.) state.

The energy source then proceeds to boot up by executing six cycles.

Each cube only ever considers its neighbors: any of the 26 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3, its neighbors include the cube at x=2,y=2,z=2, the cube at x=0,y=2,z=3, and so on.

During a cycle, all cubes simultaneously change their state according to the following rules:

*    If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
*    If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.

The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the six-cycle boot process.

For example, consider the following initial state:

```
.#.
..#
###
```

Even though the pocket dimension is 3-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1 region of the 3-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z coordinate (and the frame of view follows the active cells in each cycle):

Before any cycles:

```
z=0
.#.
..#
###
```


After 1 cycle:

```
z=-1
#..
..#
.#.

z=0
#.#
.##
.#.

z=1
#..
..#
.#.
```


After 2 cycles:

```
z=-2
.....
.....
..#..
.....
.....

z=-1
..#..
.#..#
....#
.#...
.....

z=0
##...
##...
#....
....#
.###.

z=1
..#..
.#..#
....#
.#...
.....

z=2
.....
.....
..#..
.....
.....
```

After 3 cycles:

```
z=-2
.......
.......
..##...
..###..
.......
.......
.......

z=-1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=0
...#...
.......
#......
.......
.....##
.##.#..
...#...

z=1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=2
.......
.......
..##...
..###..
.......
.......
.......
```

After the full six-cycle boot process completes, 112 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles. How many cubes are left in the active state after the sixth cycle?


In [1]:
#!fsharp
let testInput = """.#.
..#
###"""

let taskInput = """#####...
.#..##..
##.##.##
...####.
#.#...##
.##...#.
.#.#.###
#.#.#..#"""

In [1]:
#!fsharp
type Space = {
    min: int[];
    max: int[];
    dimensions: int;
    store: Map<int[],bool>
}

let parse dimensions (input:string) = 
    let mutable grid = Map.empty
    let source = input.Replace("\r", "").Split("\n")
    let pos = Array.zeroCreate dimensions
    for y in 0..(source.Length-1) do
        let row = source.[y]
        pos.SetValue(y, 1)
        for x in 0..(row.Length-1) do
            if row.[x]='#' then
                pos.SetValue(x, 0)
                grid <- grid.Add( Array.copy pos, true )
    {
        min = Array.zeroCreate dimensions;
        max = Array.init dimensions (fun i -> if i = 0 then source.[0].Length-1; elif i = 1 then  source.Length-1; else 0);
        store = grid;
        dimensions = dimensions
    }

testInput |> parse 4



min,max,dimensions,store
"[ 0, 0, 0, 0 ]","[ 2, 2, 0, 0 ]",4,"[ [System.Int32[], True], [System.Int32[], True], [System.Int32[], True], [System.Int32[], True], [System.Int32[], True] ]"


In [1]:
#!fsharp
let print (space: Space) =
    let rec inner dimension (pos: int array) =
        if dimension = 1 then
            for y in space.min.[1] .. space.max.[1] do
                pos.SetValue(y,1)
                let mutable s = ""
                for x in space.min.[0] .. space.max.[0] do                            
                    pos.SetValue(x, 0)
                    match space.store.TryFind pos with
                    | Some value when value ->
                        s <- s + "#"
                    | _ -> 
                        s <- s + "."
                printf "%s" s
        else
            let letter = 
                match dimension with
                | 2 -> "Z"
                | 3 -> "W"
                | 4 -> "T"
                | 5 -> "R"
                | index -> "S" + (index-6).ToString()
            for coord in space.min.[dimension] .. space.max.[dimension] do
                printf "%s" (sprintf "%s = %i" letter coord)
                let newPos = 
                    Array.mapi
                        (function
                         | index when index = dimension -> fun _ -> coord
                         | _ -> id
                        ) 
                        pos
                inner (dimension-1) newPos
                printfn ""
    
    inner (space.dimensions-1) space.min

testInput |> parse 3 |> print

Z = 0

.#.

..#

###

In [1]:
#!fsharp
let countNeighbors (space:Space) (coords: int array) =
    let rec inner dimension (pos:int array) allZero =
        let mutable sum = 0
        if dimension = space.dimensions - 1 then
            for i in -1..1 do
                if not (allZero && i=0) then
                    pos.SetValue(i + coords.[dimension], dimension)
                    match space.store.TryFind pos with
                    | Some value when value -> sum <- sum + 1
                    | _ -> ()
        else
            for i in -1..1 do
                pos.SetValue(i + coords.[dimension], dimension)
                sum <- sum + inner (dimension+1) pos (allZero && i=0)
        sum
    inner 0 (Array.zeroCreate space.dimensions) true                    

countNeighbors (testInput |> parse 3) [|0; 2; 0|]

1

In [1]:
#!fsharp
let step (space: Space) =
    let mutable minimum:int[] = Array.zeroCreate space.dimensions
    let mutable maximum:int[] = Array.zeroCreate space.dimensions
    let mutable grid = Map.empty

    let rec inner dimension (pos: int array) =
        let mutable modified = false
        if dimension = space.dimensions - 1 then
            for i in space.min.[dimension]-1 .. space.max.[dimension] + 1 do
                pos.SetValue(i, dimension)
                let cnt = countNeighbors space pos
                let newValue = match space.store.TryFind pos with
                               | Some value when value -> cnt = 2 || cnt = 3
                               | _ -> cnt = 3
                if newValue then
                    grid <- grid.Add( Array.copy pos, true)
                    modified <- true
                    minimum.[dimension] <- min minimum.[dimension] i
                    maximum.[dimension] <- max maximum.[dimension] i
        else
            for i in space.min.[dimension]-1 .. space.max.[dimension] + 1 do
                pos.SetValue(i, dimension)
                if inner (dimension+1) pos then
                    modified <- true
                    minimum.[dimension] <- min minimum.[dimension] i
                    maximum.[dimension] <- max maximum.[dimension] i

        modified

    inner 0 (Array.zeroCreate space.dimensions) |> ignore
    {
        min = minimum
        max = maximum
        store = grid
        dimensions = space.dimensions
    }

testInput |> parse 3 |> step |> print


Z = -1

...

#..

..#

.#.

...

Z = 1

...

#..

..#

.#.

In [1]:
#!fsharp
let simulate steps (space: Space) =
    let mutable state = space
    for i in 1..steps do
        state <- step state
    state.store.Count

testInput |> parse 3 |> simulate 6

112

In [1]:
#!fsharp
taskInput |> parse 3 |> simulate 6

313

In [1]:
#!fsharp
testInput |> parse 4 |> simulate 6

848

In [1]:
#!fsharp
taskInput |> parse 4 |> simulate 6

2640